## Biblioteke

In [1]:
import tensorflow as tf
import numpy as np

# WaveNet

#### Parametri mreže

In [70]:
learning_rate = 0.001
num_steps = 100

#### Hiperparametri

In [71]:
# treba nadodati iznose hiperparametara

In [72]:
batch_size = 5
dilation_layers = 40
dilations = [(2 ** (a % 10)) for a in range(dilation_layers)]
filter_width = 2
sequence_length = 8000
residual_channels = 32
dilation_channels = 32
quantization_channels = 256
receptive_field = sum(dilations) + 1
#use_biases = True
#skip_channels = None
#scalar_input = None
#initial_filter_width = None
#histogarms = None
#global_condition_channels = None
#global_condition_cardinality = None
#receptive_field = None

In [73]:
print(sum(dilations))

4092


#### Placeholderi

In [74]:
# Treba nadodati dimenziju plejsholdera

In [92]:
x = tf.placeholder(tf.float32, [None, sequence_length, quantization_channels])
y = tf.placeholder(tf.float32, [None, (sequence_length - receptive_field), quantization_channels])

#### Varijable

In [76]:
'''
# podfunkcija za izgradnju varijable
def create_variable(name, shape):
    initializer = tf.contrib.layers.xavier_initializer_conv2d()
    variable = tf.Variable(initializer(shape=shape), name=name)
    return variable
'''

'\n# podfunkcija za izgradnju varijable\ndef create_variable(name, shape):\n    initializer = tf.contrib.layers.xavier_initializer_conv2d()\n    variable = tf.Variable(initializer(shape=shape), name=name)\n    return variable\n'

In [77]:
'''
def build_vars(dilations):
    var = dict()
    
    # causal layer
    #TO DO
    layer = dict()
    layer['filter'] = create_variable('filter', [initial_filter_width, initial_channels, self.residual_channels])
    var['causal_layer'] = layer
    
    # dilated stack
    #TO DO
    for i, dilation in enumerate(self.dilations):
        current = dict()
        current['filter'] = create_variable('filter', [self.filter_width, self.residual_channels, self.dilation_channels])
        current['gate'] = create_variable('gate', [self.filter_width, self.residual_channels, self.dilation_channels])
        current['dense'] = create_variable('dense', [1, self.dilation_channels, self.residual_channels])
        current['skip'] = create_variable('skip', [1, self.dilation_channels, self.skip_channels])
    
        if self.use_biases:
            current['filter_bias'] = create_bias_variable('filter_bias', [self.dilation_channels])
            current['gate_bias'] = create_bias_variable('gate_bias', [self.dilation_channels])
            current['dense_bias'] = create_bias_variable('dense_bias', [self.residual_channels])
            current['skip_bias'] = create_bias_variable('slip_bias', [self.skip_channels])

        var['dilated_stack'].append(current)
        
    # postprocessing
    #TO DO
    current = dict()
    current['postprocess1'] = create_variable('postprocess1', [1, self.skip_channels, self.skip_channels])
    current['postprocess2'] = create_variable('postprocess2', [1, self.skip_channels, self.quantization_channels])
    if self.use_biases:
        current['postprocess1_bias'] = create_bias_variable('postprocess1_bias', [self.skip_channels])
        current['postprocess2_bias'] = create_bias_variable('postprocess2_bias', [self.quantization_channels])
    var['postprocessing'] = current

    return var
'''

"\ndef build_vars(dilations):\n    var = dict()\n    \n    # causal layer\n    #TO DO\n    layer = dict()\n    layer['filter'] = create_variable('filter', [initial_filter_width, initial_channels, self.residual_channels])\n    var['causal_layer'] = layer\n    \n    # dilated stack\n    #TO DO\n    for i, dilation in enumerate(self.dilations):\n        current = dict()\n        current['filter'] = create_variable('filter', [self.filter_width, self.residual_channels, self.dilation_channels])\n        current['gate'] = create_variable('gate', [self.filter_width, self.residual_channels, self.dilation_channels])\n        current['dense'] = create_variable('dense', [1, self.dilation_channels, self.residual_channels])\n        current['skip'] = create_variable('skip', [1, self.dilation_channels, self.skip_channels])\n    \n        if self.use_biases:\n            current['filter_bias'] = create_bias_variable('filter_bias', [self.dilation_channels])\n            current['gate_bias'] = create_bi

#### Mreža

In [78]:
'''
def causal_conv(x, filter_, dilation):
    pass

def create_causal(x, variables):
    weights_filter = variables['causal_layer']['filter']
    return causal_conv(x, weights_filter, 1)
    
def create_dilation(x, variables, layer_index, dilation, output_width, use_biases):
    variables = self.variables['dilated_stack'][layer_index]

    weights_filter = variables['filter']
    weights_gate = variables['gate']

    conv_filter = causal_conv(input_batch, weights_filter, dilation)
    conv_gate = causal_conv(input_batch, weights_gate, dilation)
    
    if use_biases:
            filter_bias = variables['filter_bias']
            gate_bias = variables['gate_bias']
            conv_filter = tf.add(conv_filter, filter_bias)
            conv_gate = tf.add(conv_gate, gate_bias)

    out = tf.tanh(conv_filter) * tf.sigmoid(conv_gate)
    
    ######
    #SHVATIT OD TUDA NA DOLJE
    #####################################
    
    # The 1x1 conv to produce the residual output
    weights_dense = variables['dense']
    transformed = tf.nn.conv1d(out, weights_dense, stride=1, padding="SAME", name="dense")

    # The 1x1 conv to produce the skip output
    skip_cut = tf.shape(out)[1] - output_width
    out_skip = tf.slice(out, [0, skip_cut, 0], [-1, -1, -1])
    weights_skip = variables['skip']
    skip_contribution = tf.nn.conv1d(out_skip, weights_skip, stride=1, padding="SAME", name="skip")

    if self.use_biases:
        dense_bias = variables['dense_bias']
        skip_bias = variables['skip_bias']
        transformed = transformed + dense_bias
        skip_contribution = skip_contribution + skip_bias
        
    input_cut = tf.shape(input_batch)[1] - tf.shape(transformed)[1]
    input_batch = tf.slice(input_batch, [0, input_cut, 0], [-1, -1, -1])

    return skip_contribution, input_batch + transformed
'''

'\ndef causal_conv(x, filter_, dilation):\n    pass\n\ndef create_causal(x, variables):\n    weights_filter = variables[\'causal_layer\'][\'filter\']\n    return causal_conv(x, weights_filter, 1)\n    \ndef create_dilation(x, variables, layer_index, dilation, output_width, use_biases):\n    variables = self.variables[\'dilated_stack\'][layer_index]\n\n    weights_filter = variables[\'filter\']\n    weights_gate = variables[\'gate\']\n\n    conv_filter = causal_conv(input_batch, weights_filter, dilation)\n    conv_gate = causal_conv(input_batch, weights_gate, dilation)\n    \n    if use_biases:\n            filter_bias = variables[\'filter_bias\']\n            gate_bias = variables[\'gate_bias\']\n            conv_filter = tf.add(conv_filter, filter_bias)\n            conv_gate = tf.add(conv_gate, gate_bias)\n\n    out = tf.tanh(conv_filter) * tf.sigmoid(conv_gate)\n    \n    ######\n    #SHVATIT OD TUDA NA DOLJE\n    #####################################\n    \n    # The 1x1 conv to pr

In [93]:
def build_net(inputs, filter_width, quantization_channels, dilation_channels, dilations):
    
    #print("udjen")
    
    # Input layer
    input_layer = tf.reshape(inputs, [-1, sequence_length, quantization_channels])
    
    # Izračun 
    output_width = tf.shape(inputs)[1] - receptive_field
    
    # Causal layer
    input_layer = tf.layers.conv1d(inputs=input_layer, filters=dilation_channels, kernel_size=filter_width)
    
    #print("pasan")
    
    hidden_layers = dict()
    # Hidden layers
    for i in range(dilation_layers):
        #print("brakepoint 1")
        hidden_layers[i] = dict()
        hidden_layers[i]['filter'] = tf.tanh(tf.layers.conv1d(inputs=input_layer, filters=dilation_channels, kernel_size=filter_width, dilation_rate=dilations[i]))
        #tf.Print("brakepoint 3")
        hidden_layers[i]['gate'] = tf.sigmoid(tf.layers.conv1d(inputs=input_layer, filters=dilation_channels, kernel_size=filter_width, dilation_rate=dilations[i]))
        #tf.Print("brakepoint 4")
        temp_multiplication = tf.multiply(hidden_layers[i]['filter'], hidden_layers[i]['gate'])
        #tf.Print("brakepoint 5")
        out = tf.layers.conv1d(inputs=temp_multiplication, filters=dilation_channels, kernel_size=1, padding='same')
        #print("brakepoint 6")
        input_cut = tf.shape(input_layer)[1] - tf.shape(out)[1]
        #print("brakepoint 7")
        input_layer = tf.slice(input_layer, [0, input_cut, 0], [-1, -1, -1])
        #tf.Print("brakepoint 8")
        input_layer = input_layer + out
        #print("brakepoint 9")
        skip_cut = tf.shape(out)[1] - output_width
        #print("brakepoint 10")
        #tf.Print(tf.shape(out)[1], [tf.shape(out)[1]])
        #tf.Print(output_width, [output_width])
        hidden_layers[i]['skip'] = tf.slice(out, [0, skip_cut, 0], [-1, -1, -1])
        #print("brakepoint 11")
    
    accumulation = hidden_layers[0]['skip']
    for i in range(1, dilation_layers):
        accumulation = tf.add(accumulation, hidden_layers[i]['skip'])
    
    relu1 = tf.nn.relu(accumulation)
    conv1 = tf.layers.conv1d(inputs=relu1, filters=512, kernel_size=1)
    relu2 = tf.nn.relu(conv1)
    conv2 = tf.layers.conv1d(inputs=relu2, filters=256, kernel_size=1)
    
    
    return conv2
    
    
    '''
    outputs = []
    current_layer = inputs
    
    initial_channels = 1    # treba videt parametre
    
    
    
    # causal layer
    current_layer = create_causal(current_layer)
    output_width = tf.shape(inputs)[1] - receptive_field + 1 # SHVATIT OVU LINIJU KODA
    
    # dilation layer
    for layer_index, dilation in enumerate(dilations):
        output, current_layer = create_dilation(current_layer, variables, layer_index, dilation, output_width, use_biases)
        outputs.append(output)
    
    # postprocessing
    w1 = self.variables['postprocessing']['postprocess1']
    w2 = self.variables['postprocessing']['postprocess2']
    if use_biases:
        b1 = self.variables['postprocessing']['postprocess1_bias']
        b2 = self.variables['postprocessing']['postprocess2_bias']
    
    total = sum(outputs)
    transformed1 = tf.nn.relu(total)
    conv1 = tf.nn.conv1d(transformed1, w1, stride=1, padding="SAME")
    if self.use_biases:
        conv1 = tf.add(conv1, b1)
    transformed2 = tf.nn.relu(conv1)
    conv2 = tf.nn.conv1d(transformed2, w2, stride=1, padding="SAME")
    if self.use_biases:
        conv2 = tf.add(conv2, b2)

    return conv2
    '''

#### Loss function

In [94]:
'''
def loss(input_batch, l2_regularization_strength=None):
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=target_output)
    reduced_loss = tf.reduce_mean(loss)
    return reduced_loss
'''

'\ndef loss(input_batch, l2_regularization_strength=None):\n    loss = tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=target_output)\n    reduced_loss = tf.reduce_mean(loss)\n    return reduced_loss\n'

#### Optimizer

In [95]:
# tu treba dodat optimizer
logits = build_net(x, filter_width, quantization_channels, dilation_channels, dilations)
prediction = tf.nn.softmax(logits=logits)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

#### Treniranje

In [96]:
ds = Preproc('/Users/valerio/Music/nazoF5QROxPS.128.raw', batch_size, sequence_length)
ds.preprocess()
ds.create_minibatches()

array([[[[  0,   0,   0, ...,   0,   0,   0],
         [  3,   0, 254, ..., 255,   3,   0],
         [  4,   0, 252, ..., 255,   1,   0],
         [  2,   0, 255, ..., 255,   4,   0],
         [  2,   0, 252, ..., 255,   3,   0]],

        [[  0,   0,   0, ...,   0,   0, 255],
         [  0, 254, 255, ...,   3,   0,   0],
         [  0, 252, 255, ...,   1,   0, 255],
         [  0, 255, 255, ...,   4,   0, 253],
         [  0, 252, 255, ...,   3,   0, 254]]],


       [[[255, 255,   2, ...,   0, 255, 255],
         [  0,   0,   3, ...,   0, 254, 255],
         [255, 255,   0, ...,   0, 253, 255],
         [253, 255,   6, ...,   0, 255, 255],
         [254, 255, 255, ...,   0,   0,   0]],

        [[255,   2,   0, ..., 255, 255,   3],
         [  0,   3,   0, ..., 254, 255,   4],
         [255,   0,   0, ..., 253, 255,   2],
         [255,   6,   0, ..., 255, 255,   2],
         [255, 255, 255, ...,   0,   0,   4]]]])

In [97]:
a = np.array([[0, 1, 2, 3]])
print(np.argmax(a))
e, x_lsls, y_lsls = ds.next_minibatch()

3


In [98]:
print(x_lsls.shape)

(5, 8000)


In [99]:
def one_hot(inputs, quantization_channels):
    res = np.zeros((inputs.shape[0], inputs.shape[1], quantization_channels))
    for i in range(inputs.shape[0]):
        for j in range(inputs.shape[1]):
            res[i, j, inputs[i, j]] = 1
    return res

In [100]:
import time

In [107]:
# trening sekcija
with tf.Session() as sess:
    
    # Pokretanje incijalizatora
    sess.run(init)
    
    current_epoch = 0 
    batch = 0
    
    average_loss = 0.
    average_duration = 0.
    
    while current_epoch < 10:
        
        e, batch_x, batch_y = ds.next_minibatch()
        
        x_oh = one_hot(batch_x, quantization_channels)
        y_oh = one_hot(batch_y, quantization_channels)
        y_oh = y_oh[:, receptive_field:, :]
        
        start_time = time.time()
        
        _, loss_ = sess.run([train_op, loss], feed_dict={x: x_oh, y: y_oh})
        
        duration = time.time() - start_time
        
        average_loss += loss_
        average_duration += duration
        
        print('Step:', batch, 'loss:', loss_, "duration:", duration)
        
        batch += 1
        
        if e: 
            current_epoch += 1
            print("Epoch:", current_epoch-1, "finished! Average loss is", average_loss/batch, "Average time is", average_duration/batch)
            batch = 0
            average_loss = 0.
            average_duration = 0.
    
    
    ###############################################################
    # OD TUDA NA DOLJE IDE GENERIRANJE
    ###############################################################
        
    e, batch_x, batch_y = ds.next_minibatch()
    size = batch_x.shape[1]
    x_oh = one_hot(batch_x, quantization_channels)
    x_oh = x_oh[0, :, :]
    x_oh = x_oh.reshape(1, size, quantization_channels)
    
    sound = []
    
    for i in range(16000):
        #print(i)
        #start2 = time.time()
        #start = time.time()
        predi = sess.run([prediction], feed_dict={x: x_oh})
        #print(time.time() - start)
        #start = time.time()
        x_oh[:, :-1, :] = x_oh[:, 1:, :]
        #print(time.time() - start)
        reza = predi[0]
        x_oh[:, -1, :] = reza[:, -1, :]
        sound.append(np.argmax(reza[0, -1, :]))
        #print(time.time() - start2)
    
    filebytearray = bytearray(sound)
    
    file = fopen("sound.raw", "wb")
    
    file.write(filebytearray)
    file.close()
    
    print('Optimization finished')

Step: 0 loss: 5.48986 duration: 6.868599891662598
Epoch: 0 finished! Average loss is 5.48986053467 Average time is 6.868599891662598
Step: 0 loss: 4.99703 duration: 5.303632020950317
Step: 1 loss: 4.21001 duration: 4.8583738803863525
Epoch: 1 finished! Average loss is 4.60352063179 Average time is 5.081002950668335
Step: 0 loss: 3.13336 duration: 5.565047025680542
Step: 1 loss: 2.50405 duration: 5.001895189285278
Epoch: 2 finished! Average loss is 2.81870746613 Average time is 5.28347110748291
Step: 0 loss: 2.37078 duration: 4.751143932342529
Step: 1 loss: 2.27066 duration: 5.6316819190979
Epoch: 3 finished! Average loss is 2.32072126865 Average time is 5.191412925720215
Step: 0 loss: 2.12469 duration: 5.721787929534912
Step: 1 loss: 2.14274 duration: 5.079869270324707
Epoch: 4 finished! Average loss is 2.13371419907 Average time is 5.40082859992981
Step: 0 loss: 2.04262 duration: 6.463510036468506
Step: 1 loss: 1.96071 duration: 5.165129899978638
Epoch: 5 finished! Average loss is 2.0

NameError: name 'fopen' is not defined

In [112]:
fi = open("sound.raw", "wb")

In [114]:
fi.write(filebytearray)
fi.close()